In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd

from pathlib import Path

In [3]:
train_dir = Path('E:\data\RSNA2024')

In [4]:
class CFG:
    random_seed = 42

    image_size = 256
    
    ROOT_FOLDER = train_dir / 'original'
    DEST_FOLDER = train_dir
    PNG_DIR = DEST_FOLDER / f'pngs_{image_size}'
    IMAGES_DIR = ROOT_FOLDER / 'train_images'
    TRAIN_CSV = ROOT_FOLDER / 'train.csv'
    FILES_CSV = ROOT_FOLDER / 'train_files.csv'
    TRAIN_DESC_CSV = ROOT_FOLDER / 'train_series_descriptions.csv'
    COORDS_CSV = ROOT_FOLDER / 'train_label_coordinates.csv'

### Train_df

In [5]:
train_df = pd.read_csv(CFG.TRAIN_CSV)
train_desc_df = pd.read_csv(CFG.TRAIN_DESC_CSV)

train_df.shape, train_desc_df.shape

((1975, 26), (6294, 3))

In [6]:
train_desc_df['ss_id'] = train_desc_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)

In [7]:
train_df.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [8]:
cols = train_df.columns[1:]
# first = [c.split('_')[:-2] for c in cols]
# last = [c.split('_')[-2:] for c in cols]

cols = [c.split('_') for c in cols]
cols = [''.join([i[0] if len(i) > 2 else i for i in c]).upper() for c in cols]

cols = ['study_id'] + cols

cols[:5]

['study_id', 'SCSL1L2', 'SCSL2L3', 'SCSL3L4', 'SCSL4L5']

In [9]:
dict(zip(train_df.columns, cols))

{'study_id': 'study_id',
 'spinal_canal_stenosis_l1_l2': 'SCSL1L2',
 'spinal_canal_stenosis_l2_l3': 'SCSL2L3',
 'spinal_canal_stenosis_l3_l4': 'SCSL3L4',
 'spinal_canal_stenosis_l4_l5': 'SCSL4L5',
 'spinal_canal_stenosis_l5_s1': 'SCSL5S1',
 'left_neural_foraminal_narrowing_l1_l2': 'LNFNL1L2',
 'left_neural_foraminal_narrowing_l2_l3': 'LNFNL2L3',
 'left_neural_foraminal_narrowing_l3_l4': 'LNFNL3L4',
 'left_neural_foraminal_narrowing_l4_l5': 'LNFNL4L5',
 'left_neural_foraminal_narrowing_l5_s1': 'LNFNL5S1',
 'right_neural_foraminal_narrowing_l1_l2': 'RNFNL1L2',
 'right_neural_foraminal_narrowing_l2_l3': 'RNFNL2L3',
 'right_neural_foraminal_narrowing_l3_l4': 'RNFNL3L4',
 'right_neural_foraminal_narrowing_l4_l5': 'RNFNL4L5',
 'right_neural_foraminal_narrowing_l5_s1': 'RNFNL5S1',
 'left_subarticular_stenosis_l1_l2': 'LSSL1L2',
 'left_subarticular_stenosis_l2_l3': 'LSSL2L3',
 'left_subarticular_stenosis_l3_l4': 'LSSL3L4',
 'left_subarticular_stenosis_l4_l5': 'LSSL4L5',
 'left_subarticular_ste

In [10]:
train_df.rename(columns=dict(zip(train_df.columns, cols)), inplace=True)
train_df.shape

(1975, 26)

In [11]:
train_df.iloc[0]

study_id        4003253
SCSL1L2     Normal/Mild
SCSL2L3     Normal/Mild
SCSL3L4     Normal/Mild
SCSL4L5     Normal/Mild
SCSL5S1     Normal/Mild
LNFNL1L2    Normal/Mild
LNFNL2L3    Normal/Mild
LNFNL3L4    Normal/Mild
LNFNL4L5       Moderate
LNFNL5S1    Normal/Mild
RNFNL1L2    Normal/Mild
RNFNL2L3    Normal/Mild
RNFNL3L4       Moderate
RNFNL4L5       Moderate
RNFNL5S1    Normal/Mild
LSSL1L2     Normal/Mild
LSSL2L3     Normal/Mild
LSSL3L4     Normal/Mild
LSSL4L5        Moderate
LSSL5S1     Normal/Mild
RSSL1L2     Normal/Mild
RSSL2L3     Normal/Mild
RSSL3L4     Normal/Mild
RSSL4L5     Normal/Mild
RSSL5S1     Normal/Mild
Name: 0, dtype: object

In [12]:
train_df.study_id.nunique()

1975

In [13]:
vals = {'Normal/Mild': 'N', 'Moderate': 'M', 'Severe': 'S'}
vals

{'Normal/Mild': 'N', 'Moderate': 'M', 'Severe': 'S'}

In [14]:
train_df[cols[1:]] = train_df[cols[1:]].replace(vals)

In [15]:
train_df.sample(2)

,study_id,SCSL1L2,SCSL2L3,SCSL3L4,SCSL4L5,SCSL5S1,LNFNL1L2,LNFNL2L3,LNFNL3L4,LNFNL4L5,...,LSSL1L2,LSSL2L3,LSSL3L4,LSSL4L5,LSSL5S1,RSSL1L2,RSSL2L3,RSSL3L4,RSSL4L5,RSSL5S1
1792,3885334932,N,N,N,S,N,N,N,N,M,...,N,N,M,S,N,N,N,N,S,N
303,677672203,N,N,N,N,N,N,N,M,N,...,N,M,M,M,N,N,M,M,S,M


### Coordinates_df

In [16]:
coords_df = pd.read_csv(CFG.COORDS_CSV)
files_df = pd.read_csv(CFG.FILES_CSV)

coords_df.shape, files_df.shape

((48692, 7), (147218, 11))

In [17]:
coords_df.rename(columns={'instance_number': 'instance'}, inplace=True)

In [18]:
coords_df.study_id.nunique(), coords_df.condition.nunique(), coords_df.level.nunique()

(1974, 5, 5)

In [19]:
coords_df.condition.unique(), coords_df.level.unique()

(array(['Spinal Canal Stenosis', 'Right Neural Foraminal Narrowing',
        'Left Neural Foraminal Narrowing', 'Left Subarticular Stenosis',
        'Right Subarticular Stenosis'], dtype=object),
 array(['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1'], dtype=object))

In [20]:
coords_df.series_id.nunique()

6291

In [21]:
coords_df['ss_id'] = coords_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)
coords_df['instance_id'] = coords_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}_{str(row["instance"])}', axis=1)

In [22]:
train_desc_df.sample()

,study_id,series_id,series_description,ss_id
4640,3170320453,102220493,Sagittal T1,3170320453_102220493


In [23]:
coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id
14375,1286671775,3732874112,9,Spinal Canal Stenosis,L3/L4,278.043860,259.087719,1286671775_3732874112,1286671775_3732874112_9
21668,1936447237,3308305155,10,Spinal Canal Stenosis,L3/L4,226.545455,212.909091,1936447237_3308305155,1936447237_3308305155_10


In [24]:
# rename condition
coords_df['condition'] = coords_df.apply(lambda row: ''.join([w[0] for w in row['condition'].split(' ')]), axis=1)

In [25]:
# rename level
coords_df['level'] = coords_df.level.apply(lambda l: ''.join(l.split('/')))

In [26]:
coords_df['cl'] = coords_df['condition'] + coords_df['level']

In [27]:
coords_df.condition.nunique()

5

In [28]:
coords_df.head(10)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl
0,4003253,702807833,8,SCS,L1L2,322.831858,227.964602,4003253_702807833,4003253_702807833_8,SCSL1L2
1,4003253,702807833,8,SCS,L2L3,320.571429,295.714286,4003253_702807833,4003253_702807833_8,SCSL2L3
2,4003253,702807833,8,SCS,L3L4,323.030303,371.818182,4003253_702807833,4003253_702807833_8,SCSL3L4
3,4003253,702807833,8,SCS,L4L5,335.292035,427.327434,4003253_702807833,4003253_702807833_8,SCSL4L5
4,4003253,702807833,8,SCS,L5S1,353.415929,483.964602,4003253_702807833,4003253_702807833_8,SCSL5S1
5,4003253,1054713880,4,RNFN,L4L5,187.961759,251.839388,4003253_1054713880,4003253_1054713880_4,RNFNL4L5
6,4003253,1054713880,4,RNFN,L5S1,198.240918,285.613767,4003253_1054713880,4003253_1054713880_4,RNFNL5S1
7,4003253,1054713880,5,RNFN,L3L4,187.227533,210.722753,4003253_1054713880,4003253_1054713880_5,RNFNL3L4
8,4003253,1054713880,6,RNFN,L1L2,194.569790,127.755258,4003253_1054713880,4003253_1054713880_6,RNFNL1L2
9,4003253,1054713880,6,RNFN,L2L3,191.632887,165.934990,4003253_1054713880,4003253_1054713880_6,RNFNL2L3


In [29]:
train_desc_df.series_description.unique()

array(['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2'], dtype=object)

In [30]:
train_desc_df.series_description.isna().sum()

0

In [31]:
coords_df.shape

(48692, 10)

In [32]:
coords_df = pd.merge(coords_df, train_desc_df.loc[:, ['ss_id', 'series_description']],  how='inner', left_on=['ss_id'], right_on=['ss_id'])

coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description
30387,2701226232,3116538090,13,LNFN,L1L2,292.789620,119.548322,2701226232_3116538090,2701226232_3116538090_13,LNFNL1L2,Sagittal T1
36102,3194007225,883557389,19,RSS,L4L5,234.176368,266.511115,3194007225_883557389,3194007225_883557389_19,RSSL4L5,Axial T2


In [33]:
coords_df.shape

(48692, 11)

In [34]:
# coords_df['plane'] = coords_df.apply(lambda row: train_desc_df[train_desc_df['ss_id'] == row['ss_id']].series_description.values[0], axis=1)

In [35]:
# coords_df.sample(5)

In [36]:
# check canal stenosis is noy only in axial plane
coords_df[(coords_df.condition == 'SCS') & (coords_df.series_description != 'Axial T2')].sample()

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description
3940,332284668,4277644621,12,SCS,L5S1,222.585361,308.832753,332284668_4277644621,332284668_4277644621_12,SCSL5S1,Sagittal T2/STIR


In [37]:
# get the positive slices
coords_df.groupby(['study_id','series_id']).instance.unique()

study_id    series_id 
4003253     702807833                              [8]
            1054713880               [4, 5, 6, 11, 12]
            2448190387          [3, 4, 11, 19, 28, 35]
4646740     3201256954    [15, 16, 22, 28, 29, 34, 40]
            3486248476           [5, 6, 7, 15, 16, 17]
                                      ...             
4287160193  1507070277                             [8]
            1820446240          [4, 9, 10, 16, 22, 28]
4290709089  3274612423                             [9]
            3390218084    [2, 3, 5, 6, 10, 15, 20, 21]
            4237840455                  [4, 5, 11, 12]
Name: instance, Length: 6291, dtype: object

In [38]:
coords_df.ss_id.nunique(), coords_df.instance_id.nunique()

(6291, 24546)

### Files_df

In [39]:
files_df = pd.read_csv(CFG.FILES_CSV)

In [40]:
files_df.sample(5)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription
17371,1459284649,375943350,13,-13,13,512,512,4.0,4.4,HFS,T2
77369,3039159884,4236083882,4,4,4,384,384,4.0,4.6,HFS,NaN
80433,3128832901,1312085888,21,-17,21,512,512,4.0,5.0,HFS,T2
63725,2683615288,4218080131,6,85,6,320,320,4.0,4.4,HFS,T2
68550,2802584821,829885547,1,183,1,320,320,4.0,4.4,HFS,T2


In [41]:
# files_df.rename(columns={'patient': 'study_id', 'series': 'series_id', 'image': 'instance'}, inplace=True)

In [42]:
files_df.patientposition.value_counts(), files_df.patientposition.isna().sum()

(patientposition
 HFS    118249
 FFS     28969
 Name: count, dtype: int64,
 0)

In [43]:
files_df.groupby(['study_id']).patientposition.unique().value_counts()

patientposition
[HFS]    1585
[FFS]     390
Name: count, dtype: int64

In [44]:
files_df['ss_id'] = files_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)
files_df['instance_id'] = files_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}_{str(row["instancenumber"])}', axis=1)

In [45]:
source_dir = CFG.PNG_DIR
files_df['filename'] = files_df.apply(lambda row: f'{source_dir}\\{row.study_id}_{row.series_id}_{row.image}.png', axis=1)

In [46]:
files_df.sample()

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename
61399,2622319181,3083701757,18,-41,18,760,640,3.0,3.6,HFS,T1,2622319181_3083701757,2622319181_3083701757_18,E:\data\RSNA2024\pngs_256\2622319181_308370175...


In [47]:
train_desc_df.sample()

,study_id,series_id,series_description,ss_id
5365,3664042467,3448350502,Axial T2,3664042467_3448350502


In [48]:
coords_df = pd.merge(coords_df, files_df[['instance_id', 'rows', 'columns', 'filename','patientposition']], left_on='instance_id', right_on='instance_id')

In [49]:
coords_df.shape

(48692, 15)

In [50]:
coords_df.sample()

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description,rows,columns,filename,patientposition
12574,1119763924,1930073893,14,LNFN,L3L4,262.747191,223.640449,1119763924_1930073893,1119763924_1930073893_14,LNFNL3L4,Sagittal T1,512,512,E:\data\RSNA2024\pngs_256\1119763924_193007389...,HFS


In [51]:
# TODO: make sure we match coords corectly
coords_df['x_perc'] = coords_df['x'] / coords_df['columns']
coords_df['y_perc'] = coords_df['y'] / coords_df['rows']

In [52]:
ax, non_ax = coords_df[coords_df['series_description'] == 'Axial T2'], coords_df[coords_df['series_description'] != 'Axial T2']
ax.shape, non_ax.shape, coords_df.shape

((19220, 17), (29472, 17), (48692, 17))

In [53]:
coords_df.y.min()

2.063097514340344

In [54]:
for c in [ax, non_ax]:
    print(c['x_perc'].min(), c['y_perc'].min())
    print(c['x_perc'].max(), c['y_perc'].max())
    print('/////////////')
    # print(c['x_perc'].mean(), c['y_perc'].mean())

0.2849557522123894 0.3092621664050236
0.6631964653474212 0.8078096961632665
/////////////
0.0048828125 0.0040294873326959845
0.7684669901065448 0.9117647058823529
/////////////


In [55]:
files_df.shape

(147218, 14)

In [56]:
files_df = pd.merge(files_df, train_desc_df.loc[:, ['ss_id', 'series_description']],  how='inner', left_on=['ss_id'], right_on=['ss_id'])

files_df.sample(2)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename,series_description
141975,883950312,2546260085,8,-12,8,512,512,4.0,4.8,HFS,NaN,883950312_2546260085,883950312_2546260085_8,E:\data\RSNA2024\pngs_256\883950312_2546260085...,Sagittal T1
116895,4061375549,3611950405,11,-376,11,448,448,4.0,4.8,HFS,T2,4061375549_3611950405,4061375549_3611950405_11,E:\data\RSNA2024\pngs_256\4061375549_361195040...,Axial T2


In [57]:
files_df.shape

(147218, 15)

In [58]:
files_df['cl'] = 'H'
files_df['condition'] = 'H'

In [59]:
files_df.sample(5)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename,series_description,cl,condition
45058,2206871806,179530305,58,91,58,512,512,2.8,1.40,HFS,T2,2206871806_179530305,2206871806_179530305_58,E:\data\RSNA2024\pngs_256\2206871806_179530305...,Axial T2,H,H
120552,41477684,2595734107,8,15,8,512,512,4.0,5.00,FFS,T2,41477684_2595734107,41477684_2595734107_8,E:\data\RSNA2024\pngs_256\41477684_2595734107_...,Sagittal T2/STIR,H,H
90808,3367650254,1536898889,3,-485,3,320,320,3.5,3.50,HFS,T2,3367650254_1536898889,3367650254_1536898889_3,E:\data\RSNA2024\pngs_256\3367650254_153689888...,Axial T2,H,H
141684,870699023,1241187576,42,3,42,576,576,4.0,4.40,HFS,NaN,870699023_1241187576,870699023_1241187576_42,E:\data\RSNA2024\pngs_256\870699023_1241187576...,Axial T2,H,H
138564,786544183,410008846,6,27,6,448,448,4.0,4.48,HFS,T2,786544183_410008846,786544183_410008846_6,E:\data\RSNA2024\pngs_256\786544183_410008846_...,Sagittal T2/STIR,H,H


### Save results

In [60]:
train_df.to_csv(CFG.DEST_FOLDER / 'train.csv', index=False)

In [61]:
coords_df.to_csv(CFG.DEST_FOLDER / 'train_label_coordinates.csv', index=False)

In [62]:
files_df.to_csv(CFG.DEST_FOLDER / 'train_files.csv', index=False)